In [1]:
from langchain_community.retrievers import BM25Retriever
from anthropic import Anthropic
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatAnthropic, ChatOllama, ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS, Chroma
import keyring

# api key
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
ANTHROPIC_API_KEY = keyring.get_password('anthropic', 'key_for_windows')

In [2]:
model = 'gpt-4o'

data = [
    {
        "기업명": "삼성전자",
        "날짜": "2024-03-02",
        "문서 카테고리": "인수합병",
        "요약": "삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.",
        "주요 이벤트": ["기업 인수합병"]
    }, {
        "기업명": "삼성전자",
        "날짜": "2024-03-24",
        "문서 카테고리": "인수합병",
        "요약": "테스트 하나 둘 셋",
        "주요 이벤트": ["신제품 출시"]
    }, {
        "기업명": "현대차",
        "날짜": "2024-04-02",
        "문서 카테고리": "인수합병",
        "요약": "삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.",
        "주요 이벤트": ["기업 인수합병", "신제품 출시"]
    }
]

# data 내에 '요약'만 추출해서 리스트로 반환
doc_list = [item['요약'] for item in data]

# elastic search based retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list, metadatas=[{"source": i} for i in range(len(data))]
)
bm25_retriever.k = 1

# embedding from openai
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# vector database retriever
## FAISS
faiss_vectorstore = FAISS.from_texts(
    doc_list, embedding=embedding, metadatas=[{'source': i} for i in range(len(data))]
)
faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={'k':1}
)
## Chroma
chroma_vectorstore = Chroma.from_texts(
    doc_list, embedding=embedding, metadatas=[{'source': i} for i in range(len(data))]
)
chroma_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={'k':1}
)

# ensemble retreiver
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever, chroma_retriever],
    weights=[0.2, 0.5, 0.3]
)

def search(query):
    ensemble_docs = ensemble_retriever.invoke(query)
    return ensemble_docs
    
def get_answer(query, model):
    if 'gpt' in model:
        llm = OpenAI(api_key=OPENAI_API_KEY)
        response = llm.chat.completions.create(
            model=model,
            messages=[{
                'role':'system',
                'content':'You are a helpful assistant.'
            }, {
                'role':'user',
                'content':query
            }]
        )
        answer = response.choices[0].message.content
    elif 'claude' in model:
        llm = Anthropic(api_key=ANTHROPIC_API_KEY)
        response = llm.messages.create(
            model=model,
            max_tokens=1024,
            messages=[
                {'role':'user', 'content':query},
                {'role':'assistant', 'content':"Hello!"},
            ]
        )
        answer = response.content[0].text
        
    elif 'llama' in model:
        llm = ChatOllama(model=model)
        response = llm.invoke(query)
        answer = response.content
        
    else:
        print("Not supported")
        return None
    
    return answer

def prompt_and_generate(query, docs, model):
    
    prompt = f"""아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    
    # 질문 : {query}
    
    # 뉴스
    """
    
    # docs가 여러 개인 경우 나눠서 붙임
    for i in range(len(docs)):
        prompt += f"뉴스{i+1}\n"
        prompt += f"요약: {docs[i].page_content}\n"
        idx = docs[i].metadata['source']
        prompt += f"카테고리: {data[idx]['문서 카테고리']}\n"
        prompt += f"이벤트: {data[idx]['주요 이벤트']}\n"
        prompt += '\n'
        
    print(prompt)
    answer = get_answer(prompt, model=model)
    return answer
        

In [3]:
query = "삼성전자가 인수하는 기업은?"
retrieved = [doc for doc in search(query)]
answer = prompt_and_generate(query, docs=retrieved, model='claude-3-5-sonnet-20240620')
print(answer)

아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    
    # 질문 : 삼성전자가 인수하는 기업은?
    
    # 뉴스
    뉴스1
요약: 삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.
카테고리: 인수합병
이벤트: ['기업 인수합병']

뉴스2
요약: 테스트 하나 둘 셋
카테고리: 인수합병
이벤트: ['신제품 출시']


!! 질문과 가장 적절한 뉴스를 선택하고 답변을 생성해 드리겠습니다.

질문: 삼성전자가 인수하는 기업은?

가장 적절한 뉴스: 뉴스1

답변: 삼성전자는 현재 HVAC(냉난방공조) 사업 인수를 타진 중입니다. 이는 삼성전자가 기존 가전 사업의 약점을 보완하기 위한 전략적 움직임으로 보입니다. 구체적인 기업명은 언급되지 않았지만, HVAC 분야의 기업을 인수할 계획인 것으로 보입니다.

이벤트: 기업 인수합병


In [4]:
answer = prompt_and_generate(query, docs=retrieved, model='gpt-4o')
print(answer)

아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    
    # 질문 : 삼성전자가 인수하는 기업은?
    
    # 뉴스
    뉴스1
요약: 삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.
카테고리: 인수합병
이벤트: ['기업 인수합병']

뉴스2
요약: 테스트 하나 둘 셋
카테고리: 인수합병
이벤트: ['신제품 출시']


# 삼성전자가 인수하는 기업은?

- **답변**: 삼성전자가 인수하려고 하는 기업은 HVAC(냉난방공조) 사업에 관련된 기업입니다.

- **뉴스에서 등장한 이벤트**: 기업 인수합병

**참고 뉴스 요약:**
삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.


In [5]:
answer = prompt_and_generate(query, docs=retrieved, model='llama3.1')
print(answer)

아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오. 질문과 가장 적절한 뉴스에서 등장한 이벤트까지 출력하세요.
    
    # 질문 : 삼성전자가 인수하는 기업은?
    
    # 뉴스
    뉴스1
요약: 삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.
카테고리: 인수합병
이벤트: ['기업 인수합병']

뉴스2
요약: 테스트 하나 둘 셋
카테고리: 인수합병
이벤트: ['신제품 출시']


Samsung Electronics는 HVAC 사업에 적극 투자하고 있다. 

*   뉴스1의 '기업 인수합병'과 같은 맥락으로, 삼성전자가 HVAC(냉난방공조) 사업을 인수하는 예시입니다.
*   뉴스2의 '신제품 출시'와는 전혀 관련이 없습니다.

  Samsung Electronics가 인수하는 기업은 HVAC(냉난방공조) 사업에 있습니다.
